In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(12,5)})
%load_ext autoreload
%autoreload 2

In [162]:
from tqdm import tqdm

In [2]:
import requests

In [38]:
with open('toekn.txt', 'r') as f:
    API_TOKEN = f.readlines()[0][:-1]

In [40]:
import berserk

session = berserk.TokenSession(API_TOKEN)
client = berserk.Client(session=session)

In [41]:
berserk.enums.PerfType.BLITZ

'blitz'

In [98]:
def color2bool(color):
    if color == 'white':
        return True
    else:
        return False

def bool2color(b):
    if b:
        return 'white'
    else:
        return 'black'

def flip_color(color):
    if color == 'white':
        return 'black'
    else:
        return 'white'

In [85]:
def my_color(playes_data):
    if playes_data['white']['user']['name'] == 'menisadi':
        return 'white'
    else:
        return 'black'

In [118]:
def opponent_name(playes_data):
    return playes_data[flip_color(my_color(playes_data))]['user']['name']

In [158]:
    games = client.games.export_by_player(username='menisadi', perf_type='blitz', 
                                          rated=True, analysed=True, opening=True, 
                                          evals=True)

## One game

In [57]:
first_game = next(games)

In [155]:
playes = first_game['players']

meni_color = my_color(playes)

my_rating = first_game['players'][meni_color]['rating']
opponent_rating = first_game['players'][flip_color(meni_color)]['rating']
opponent = opponent_name(first_game['players'])

time = first_game['createdAt']

termination = first_game['status']

winner = first_game['winner']
did_i_win = winner == meni_color

openning = first_game['opening']['name']

moves = first_game['moves']
game_length = len(moves.split(' '))

my_accuracy = first_game['players'][meni_color]['analysis']['acpl']
opponent_accuracy = first_game['players'][flip_color(meni_color)]['analysis']['acpl']

In [157]:
new_row = pd.DataFrame([{
        'time': time,
        'opponent_name': opponent,
        'my_rating': my_rating,
        'my_color': meni_color,
        'result': winner,
        'did_I_win': did_i_win, 
        'termination_type': termination,
        'length_of_game': game_length,
        'my_average_centripawn_loss': my_accuracy,
        'opponent_average_centripawn_loss': opponent_accuracy
}])

## And now we loop

In [159]:
df = pd.DataFrame(
    columns=['time', 'opponent_name', 'my_rating', 'my_color', 
             'result', 'termination_type', 'length_of_game', 
             'my_average_centripawn_loss', 'opponent_average_centripawn_loss', 'opening'])

In [164]:
limit = 3

for game in tqdm(games):
    playes = game['players']

    meni_color = my_color(playes)
    my_rating = game['players'][meni_color]['rating']
    opponent_rating = game['players'][flip_color(meni_color)]['rating']
    opponent = opponent_name(game['players'])
    
    time = game['createdAt']
    termination = game['status']
    winner = game['winner']
    did_i_win = winner == meni_color
    openning = game['opening']['name']
    moves = game['moves']
    game_length = len(moves.split(' '))
    my_accuracy = game['players'][meni_color]['analysis']['acpl']
    opponent_accuracy = game['players'][flip_color(meni_color)]['analysis']['acpl']

    new_row = pd.DataFrame([{
            'time': time,
            'opponent_name': opponent,
            'my_rating': my_rating,
            'my_color': meni_color,
            'result': winner,
            'did_I_win': did_i_win, 
            'termination_type': termination,
            'length_of_game': game_length,
            'my_average_centripawn_loss': my_accuracy,
            'opponent_average_centripawn_loss': opponent_accuracy
    }])

    pd.concat([df, new_row])

    i -= 1
    if i == 0:
        break


0it [00:00, ?it/s]
